# Pseudo dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblbagging/pseudodata_claude.ipynb)

In [1]:
%%capture
%pip install iblm --upgrade

In [2]:
import os
import pandas as pd
import numpy as np

from iblm import IBLModel

from sklearn.datasets import make_classification

#os.environ["ANTHROPIC_API_KEY"] = "YOUR_API_KEY"

### Preparing data sets

In [3]:
num_train_data = 500 # 50〜300
num_test_data = 1000
sample = num_train_data + num_test_data
n_informative = 2
n_redundant = 0
n_features = n_informative + n_redundant
weights = [0.5, 0.5]
flip_y=0
seed = 3655  # 3655,3656,3657

x, y = make_classification(
    n_samples = sample,
    n_features = n_features,
    n_informative = n_informative,
    n_redundant = n_redundant,
    weights = weights,
    flip_y = flip_y,
    random_state = seed
    )

x = np.round(x, decimals=3)
x = pd.DataFrame(x)

x_test = x[0:num_test_data]
x_train = x[num_test_data:]

y_test = y[:num_test_data]
y_train = y[num_test_data:]

### IBL setting

In [4]:
iblm = IBLModel(
    api_type="claude",
    model_name="claude-3-opus-20240229",
    objective="binary"
    )

* Learning

In [5]:
model = iblm.fit(x_train, y_train)
print(model)

2024-03-10 18:34:59,112 [iblm.ibl][INFO] (ibl:ibl.py:fit:153)


#########
def predict(x):
    import numpy as np

    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.
        x1 = row[0]
        x2 = row[1]
        
        if x1 > 0 and x2 < 0:
            y = 1.0
        elif x1 < 0 and x2 > 0:
            y = 0.0
        else:
            if x1 > 0.5 and x2 > 0.5:
                y = 0.9
            elif x1 < -0.5 and x2 < -0.5:
                y = 0.1  
            elif x1 > 0.5 and x2 < -0.5:
                y = 0.8
            elif x1 < -0.5 and x2 > 0.5:
                y = 0.2
            elif abs(x1) <= 0.5 and abs(x2) <= 0.5:
                y = 0.5
            elif x1 > 0 and x2 > 0:
                y = 0.7
            elif x1 < 0 and x2 < 0:  
                y = 0.3
            else:
                y = 0.6

        # Do not change the code after this point.
        output.append(y)
    retu

* prediction

In [6]:
y_pred = iblm.predict(x_test)

* Evaluation

In [7]:
iblm.evaluate(y_test, y_pred)

{'roc_auc': 0.990933,
 'pr_auc': 0.984234,
 'accuracy': 0.978,
 'recall': 0.970238,
 'precision': 0.985887,
 'f1_score': 0.978}